### Data Augmentation으로 비정형 데이터의 증강

In [1]:
from PIL import Image
import glob

In [2]:
# Folder Name
dirNames = ['Aiden', 'Andrew', 'Cathy']

In [3]:
# 작업 폴더 생성
import os
os.mkdir("./Face")
os.mkdir("./Face/Data")
for name in dirNames:
    os.mkdir(f"./Face/Data/{name}")

In [9]:
# Ratation과 Flip으로 데이터 증강(원본데이터 + Rotation Data + Flip Data)

for name in dirNames:
    for file in sorted(glob.glob(f"/kaggle/input/data-face/{name}/image*.jpg")):
        imgData = Image.open(file)
        # 원본 데이터 저장
        imgData.save(f"./Face/Data/{name}/{file.split('/')[-1]}") # 원본데이터는 원래아름으로 정의 
        counter = 0
        for angle in range(-15, 15, 1):
            counter+=1
            imgData2 = imgData.rotate(angle)
            imgData2.save(f"./Face/Data/{name}/rotation_{counter:03d}.png")

            imgData3 = imgData2.transpose(Image.FLIP_LEFT_RIGHT)
            imgData3.save(f"./Face/Data/{name}/flip_{counter:03d}.png")

In [7]:
# Ex : file name 정의

str = "/kaggle/input/data-face/Aiden/image_0000.jpg"
str.split("/")[-1]

'image_0000.jpg'

### 3명의 얼굴을 학습하여 인식하기

In [ ]:
# Module
import numpy as np
from PIL import Image
import glob

---
### 전체 사진중 최대 해상도 찾기

In [ ]:
dirNames = ['Aiden', 'Andrew', 'Cathy']

In [ ]:
widthRatio = []
heightRatio = []

for name in dirNames:
    for file in sorted(glob.glob(f"/kaggle/input/data-face/{name}/image*.jpg")):
        img = np.array(Image.open(file))
        widthRatio.append(img.shape[1])
        heightRatio.append(img.shape[0])

In [ ]:
print('너비 최대 해상도 :', np.max(widthRatio))
print('높이 최대 해상도 :', np.max(heightRatio))

> 해상도의 크기를 400 X 300

---
### 전체 사진을 흑백으로 변경하고 검은색 배경(400X300)의 중앙에 일치 시켜 저장하기 

In [ ]:
# 작업 폴더 생성 
import os
os.mkdir("./Face")
os.mkdir("./Face/Gray")

In [ ]:
# 이름별 directory 생성

for name in dirNames:
    os.mkdir(f"./Face/Gray/{name}")

In [ ]:
for name in dirNames:
    fileCount = 0
    for file in sorted(glob.glob(f"/kaggle/input/data-face/{name}/image*.jpg")):
        img = Image.open(file)
        imgResize = img.convert('L')
        imgArray = np.array(imgResize)

        imgDummy = np.zeros(400*300).reshape(400, 300)

        rowNum = (400 - imgArray.shape[0]) / 2
        colNum = (300 - imgArray.shape[1]) / 2

        k = 0
        for i in list(range(int(rowNum), int(rowNum) + imgArray.shape[0])):
            l = 0
            for j in list(range(int(colNum), int(colNum) + imgArray.shape[1])):
                imgDummy[i,j] = imgArray[k,l]
                l +=1
            k +=1
        img2 = Image.fromarray(imgDummy.astype('uint8'), 'L')
        img2.save(f"/kaggle/working/Face/Gray/{name}/image_{fileCount:04d}.jpg","JPEG")
        fileCount +=1

### 사진들을 numpy배열을 이용하여 Training Data 만들기

In [ ]:
number_of_data = 18 * len(dirNames)
img_width_size = 300
img_height_size = 400

In [ ]:
train_data = np.zeros(number_of_data * img_width_size * img_height_size).reshape(number_of_data, img_height_size, img_width_size)
i = 0

for name in dirNames:
    for file in sorted(glob.glob(f"/kaggle/working/Face/Gray/{name}/*.jpg")):
        img = np.array(Image.open(file))
        train_data[i,:,:] = img
        i+=1

In [ ]:
train_data.shape

### 이미지 확인

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(train_data[20], cmap='gray')
plt.show()

In [ ]:
# 여러개의 이미지를 같이 보기
plt.figure(figsize=(20, 20))
orderNo = list(range(0, len(dirNames)*18, 18))

for i in list(range(1, len(dirNames)+1)):
    plt.subplot(1, len(dirNames), i)
    plt.imshow(train_data[orderNo[i-1]].reshape(400, 300), cmap='gray')
    plt.title(dirNames[i-1])



### Target Data만들기

In [ ]:
# a = [0 for _ in range(18)]
# b = [1 for _ in range(18)]
# c = [2 for _ in range(18)]
# target_data = a + b + c

target_data = [num for num in range(3) for _ in range(18)]
print(target_data)

---
### Data들을 Tensor로 변환

In [ ]:
import torch

train_input = torch.tensor(train_data)
train_target = torch.tensor(target_data)

print(train_input.data.shape)
print(train_target.data.shape)

### train data를 훈련데이터와 검증데이터로 나누기 (정규화, 채널 추가)

In [ ]:
from sklearn.model_selection import train_test_split

train_input = train_input.data.unsqueeze(1).float() / 255.0 # 채널 차원 및 정규화
train_input, val_input, train_target, val_target = train_test_split(
                                                        train_input,
                                                        train_target,
                                                        test_size=0.2,
                                                        random_state=42
)

In [ ]:
# Dimension 확인
print(train_input.shape, train_target.shape)
print(val_input.shape, val_target.shape)

--- 
### CNN 신경망 만들기

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# Dataset 및 DataLoader 생성
batch_size = 32 # mini batch
train_dataset = TensorDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

### 모델 정의

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 100 * 75, 128) # 64, (400/4), (300/4)
        self.relu3 = nn.ReLU()
        # self.dropout = nn.Dropout(0.9)
        self.fc2 = nn.Linear(128, 3)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        # x = self.dropout(x)
        x = self.fc2(x)
        return x 

In [ ]:
# 모델, 손실함수, 옵티마이저 초기화
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
# 학습 함수
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad() # 이전 반복에서 계산된 그레디언트를 초기화 
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward() # 손실에 대한 그레디언트를 계산하고 역전파 
        optimizer.step() 
    return loss.item()

In [ ]:
# 평가함수
def evaluate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return total_loss / len(val_loader), correct / total

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model.to(device)

In [ ]:
# 훈련 반복

train_loss_scores = [] # train score list
val_loss_scores = [] # test score list

num_epochs = 50

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)
    train_loss_scores.append(train_loss)
    val_loss_scores.append(val_loss)
    print(f'Epoch : [{epoch+1} / {num_epochs}], Train Loss : {train_loss:.4f}, Val Loss : {val_loss:.4f}, Val Accuracy : {val_accuracy:.4f}')

In [ ]:
# 훈련 데이터로 평가
train_loss, train_accuracy = evaluate(model, train_loader, criterion, device)
print(f'Training Loss : {train_loss:.4f}, Training Accuracy : {train_accuracy:.4f}')

In [ ]:
# 검증 데이터로 평가
val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)
print(f'Validation Loss : {val_loss:.4f}, Validation Accuracy : {val_accuracy:.4f}')

In [ ]:
# 시각화 해보기

plt.plot(train_loss_scores)
plt.plot(val_loss_scores)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','test'])
plt.show()

---
### 이미지 불러와서 예측해 보기

In [ ]:
# Image
img = Image.open("/kaggle/working/Face/Gray/Cathy/image_0000.jpg")
img

In [ ]:
# image를 numpy 배열로 변환
img = np.array(img)
img.shape

In [ ]:
# numpy배열을 torch 변환
img = torch.from_numpy(img)
img.shape

In [ ]:
# 정규화 시키기
img = img.unsqueeze(0).float() / 255.0
img.shape

In [ ]:
# Class들의 이름 정의
classes = ['Aiden', 'Andrew', 'Cathy']
classes

In [ ]:
# 예측 함수
def predict_single_image(model, image, device, classes):
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        output = model(image.unsqueeze(0))
        _, predicted = torch.max(output, 1)
        predicted_class = classes[predicted.item()]
    return predicted_class

In [ ]:
# 예측 수행
predict_single_image(model, img, device, classes)

---
### 학습한 모델 저장하기

In [ ]:
# 전체 모델 저장
torch.save(model, './cnn_3_persons.pth')

In [ ]:
# 전체 모델 불러오기
model1 = torch.load("./cnn_3_persons.pth", weights_only=False)
model1.eval()

In [ ]:
predict_single_image(model1, img, device, classes)